读取图片

In [4]:
from datasets import load_from_disk
import os
from PIL import Image

flower_train = load_from_disk(r'/root/autodl-tmp/dataset/oxford_flower')
flower_train = flower_train['train']

butterfly_train = load_from_disk(r'/root/autodl-tmp/dataset/smithsonian_butterfly')
butterfly_train = butterfly_train['train']

oxford_generated_dir = r'/root/autodl-tmp/generated_img/oxford_flower'
butterfly_generated_dir = r'/root/autodl-tmp/generated_img/smithsonian_butterfly'

def get_generate_image(generate_image_dir):
    tmp = []
    for current_dir in os.listdir(generate_image_dir):
        tmp.append(os.path.join(generate_image_dir, current_dir))
    generate_image_dir = sorted(tmp, key=lambda x: int(x.split('/')[-1]))

    generate_image_list = []
    for current_dir in generate_image_dir[-12:]:
        for current_image_name in os.listdir(current_dir):
            generate_image_list.append(Image.open(os.path.join(current_dir, current_image_name)))

    return generate_image_list

generate_flower = get_generate_image(oxford_generated_dir)
generate_butterfly = get_generate_image(butterfly_generated_dir)

辅助函数

In [5]:
import os
import shutil

def merge_copy(src, dst):
    # 确保源目录存在
    if not os.path.exists(src):
        print(f"Error: {src} does not exist!")
        return
    
    # 如果目标目录不存在，则创建它
    if not os.path.exists(dst):
        os.makedirs(dst)

    # 列出源目录中的所有文件和文件夹
    for item in os.listdir(src):
        src_item = os.path.join(src, item)
        dst_item = os.path.join(dst, item)

        # 如果是文件，则直接复制
        if os.path.isfile(src_item):
            shutil.copy(src_item, dst_item)
        
        # 如果是目录，则递归调用此函数
        elif os.path.isdir(src_item):
            merge_copy(src_item, dst_item)

# 示例用法
# src_folder = 'path_to_source_folder'
# dst_folder = 'path_to_destination_folder'
# merge_copy(src_folder, dst_folder)


创建数据集

In [6]:
import shutil
from tqdm import tqdm

generate_dataset_dir = r'/root/autodl-tmp/generate_dataset'

if os.path.exists(generate_dataset_dir):
    shutil.rmtree(generate_dataset_dir)

def create_dataset_dir(dataset_name):
    main_dir = os.path.join(generate_dataset_dir, dataset_name)
    train_dir = os.path.join(main_dir, 'train')
    test_dir = os.path.join(main_dir, 'test')
    os.makedirs(os.path.join(train_dir, 'oxford_flower'))
    os.makedirs(os.path.join(train_dir, 'butterfly'))
    os.makedirs(os.path.join(test_dir, 'oxford_flower'))
    os.makedirs(os.path.join(test_dir, 'butterfly'))
    
create_dataset_dir('origin')
create_dataset_dir('synthetic')
create_dataset_dir('mix')

num_of_img = 200

# 真实图片 测试集
for i in tqdm(range(num_of_img)):
    flower_train[i]['image'].save(f'/root/autodl-tmp/generate_dataset/origin/test/oxford_flower/{i}.png')
    butterfly_train[i]['image'].save(f'/root/autodl-tmp/generate_dataset/origin/test/butterfly/{i}.png')

merge_copy('/root/autodl-tmp/generate_dataset/origin/test',
                     '/root/autodl-tmp/generate_dataset/mix/test/')
merge_copy('/root/autodl-tmp/generate_dataset/origin/test/',
                     '/root/autodl-tmp/generate_dataset/synthetic/test/')

# 真实图片 训练集
for i in tqdm(range(num_of_img, num_of_img * 2)):
    flower_train[i]['image'].save(f'/root/autodl-tmp/generate_dataset/origin/train/oxford_flower/{i}.png')
    butterfly_train[i]['image'].save(f'/root/autodl-tmp/generate_dataset/origin/train/butterfly/{i}.png')

merge_copy('/root/autodl-tmp/generate_dataset/origin/train',
                     '/root/autodl-tmp/generate_dataset/mix/train/')

# 合成图片
for i in tqdm(range(num_of_img)):
    generate_flower[i].save(f'/root/autodl-tmp/generate_dataset/synthetic/train/oxford_flower/{i}.png')
    generate_butterfly[i].save(f'/root/autodl-tmp/generate_dataset/synthetic/train/butterfly/{i}.png')

merge_copy('/root/autodl-tmp/generate_dataset/synthetic/train/',
                     '/root/autodl-tmp/generate_dataset/mix/train/')

  0%|          | 0/200 [00:00<?, ?it/s]

100%|██████████| 200/200 [00:02<00:00, 87.45it/s]
